In [ ]:
import os
from pathlib import Path
import shutil
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random


In [ ]:
from source.visual_genome_meta_data import read_json_to_dict
from source.visual_genome_meta_data import get_image_meta_data
from source.visual_genome_meta_data import count_occurrences
from source.visual_genome_to_yolo import create_class_mapping_from_list
from source.visual_genome_to_yolo import save_class_map_to_yaml
from source.visual_genome_to_yolo import convert_single_image_to_yolo
from source.visual_genome_to_yolo import read_yaml_to_class_map
from source.visual_genome_to_yolo import read_yolo_metadata
from source.visual_genome_to_yolo import visual_genome_to_yolo_data_n
from source.visual_genome_meta_data import plot_image_with_multiple_bboxes
from source.visual_genome_meta_data import get_image_ids
from source.yolo_training_structure import distribute_train_val_files as dist_train_val


### Define paths: 

In [ ]:
root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project')
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/test_yolo_object_train')
data_path = root_path / 'visual_genome_data'
#data_path = root_path / 'visual_genome_data_all'
yolo_path = root_path / 'visual_genome_yolo'
#yolo_path = root_path / 'visual_genome_yolo_all'

In [ ]:
yolo_path

### Read in objects file with meta data about visual genome data: 

In [ ]:
objects_file_path = data_path/'objects.json'


In [ ]:
objects = read_json_to_dict(objects_file_path)

### Get image identifiers: 

In [ ]:
image_id_list = get_image_ids(data_path)
image_id_list.sort()
len(image_id_list)

In [ ]:
image_id_list[0:3]

In [ ]:
len(objects)

In [ ]:
objects[132]

### Choose the desired objects:

In [ ]:
#desired_objects = ['forest', 'mountain', 'mountains', 'building', 'house', 
#                   'church', 'city', 'village', 'lake', 'river', 'stream', 'glacier']

#desired_objects = ['mountain']
#desired_objects = ['church']
desired_objects = ['lighthouse']


desired_objects

### Create class map based on desired objects: 

In [ ]:
class_map = create_class_mapping_from_list(desired_objects)

In [ ]:
class_map

In [ ]:

file_path = str(yolo_path) + '/'

output_path = file_path + 'class_map.yaml'
save_class_map_to_yaml(class_map, output_path)

### Check content of class_map.yaml file:

In [ ]:
file_list = os.listdir(yolo_path)
for filename in file_list:
    if filename.split('_')[-1] == 'map.yaml':
        yaml_file_name = filename
yaml_file_name

In [ ]:
yaml_path = yolo_path/yaml_file_name

class_map = read_yaml_to_class_map(str(yaml_path))

# Print the class mapping
print(class_map)

In [ ]:
len(image_id_list)

In [ ]:
#convert_single_image_to_yolo(objects[0], class_map, data_path, yolo_path)

### Create yolo compatible meta data files (bounding box information) for images containing the desired object:

In [ ]:

objects_and_ids = (objects, desired_objects, image_id_list)
paths = (data_path, yolo_path)

label_paths_w, occurrence_counts = visual_genome_to_yolo_data_n(objects_and_ids, paths, class_map)
len(label_paths_w)

In [ ]:
occurrence_counts

In [ ]:
label_paths_w[0:3]

In [ ]:
class_map

In [ ]:
label_paths_w[0:3]

### Get number of required images without desired object for balanced data set:

In [ ]:
desired_objects

In [ ]:
desired_objects[0]

In [ ]:
if len(desired_objects) == 1:
    number_occurrences = occurrence_counts[desired_objects[0]]
    print(number_occurrences)
else:
    number_occurrences = 'No unique answer: more than one desired objects!'
    print(number_occurrences)

In [ ]:
number_images_without = round((number_occurrences/100) * 30)
number_images_without

### Create meta data text files for images without desired object:

In [ ]:
label_paths_n, occurrence_counts = visual_genome_to_yolo_data_n(objects_and_ids, paths, class_map,
                                                           with_class = False, number_of_images = number_images_without)
len(label_paths_n)

In [ ]:
label_paths_n[0:3]

### The paths to the meta data files contain the image ids defining the image files to be used

#### Paths to label files with desired objects:

In [ ]:
# Paths to label files with desired objects:
print(type(label_paths_w))
print(len(label_paths_w))

#### Paths to label files without desired objects:

In [ ]:
# Paths to label files without desired objects: 
print(type(label_paths_n))
print(len(label_paths_n))

In [ ]:
label_paths = label_paths_w + label_paths_n
len(label_paths)

### Plot first three images and use class_map file to plot bounding boxes:

In [ ]:
for label_path in label_paths[0:3]:
    print(label_path)
    img_id = label_path.split('_')[-1].split('.')[0]
    print(img_id)
    
    labels, bboxes = read_yolo_metadata(label_path, class_map)
    class_names = list(labels)
    image_path_gen = data_path/'visual_genome_'
    image_path = str(image_path_gen) + str(img_id) + '.jpg'
    print(image_path)
    plot_image_with_multiple_bboxes(image_path, bboxes, class_names)
    

In [ ]:
root_path

### Create file structure to train for recognition of desired object class:

In [ ]:
train_val_trial_path = root_path / 'yolo_object_train'


if not os.path.exists(train_val_trial_path):
    os.makedirs(train_val_trial_path)

In [ ]:
train_val_trial_path

In [ ]:
train_data_path = train_val_trial_path / 'train'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)


In [ ]:
os.getcwd()

In [ ]:
train_imgages_path = train_data_path / 'images'

if not os.path.exists(train_imgages_path):
    os.makedirs(train_imgages_path)

train_labels_path = train_data_path / 'labels'

if not os.path.exists(train_labels_path):
    os.makedirs(train_labels_path)


In [ ]:
val_data_path = train_val_trial_path  / 'val'
if not os.path.exists(val_data_path):
    os.makedirs(val_data_path)

os.listdir(val_data_path)

In [ ]:
val_imgages_path = val_data_path / 'images'
if not os.path.exists(val_imgages_path):
    os.makedirs(val_imgages_path)

val_labels_path = val_data_path / 'labels'
if not os.path.exists(val_labels_path):
    os.makedirs(val_labels_path)


In [ ]:
train_images_path = train_data_path / 'images'
if not os.path.exists(train_images_path):
    os.makedirs(train_images_path)

train_labels_path = train_data_path / 'labels'
if not os.path.exists(train_labels_path):
    os.makedirs(train_labels_path)


In [ ]:
val_images_path = val_data_path / 'images'
if not os.path.exists(val_images_path):
    os.makedirs(val_images_path)

val_labels_path = val_data_path / 'labels'
if not os.path.exists(val_labels_path):
    os.makedirs(val_labels_path)


In [ ]:
train_images_grey_path = train_val_trial_path / 'train_grey/images'
 
if not os.path.exists(train_images_grey_path):
    os.makedirs(train_images_grey_path)

In [ ]:
val_images_grey_path = train_val_trial_path / 'val_grey/images'

if not os.path.exists(val_images_grey_path):
    os.makedirs(val_images_grey_path)

In [ ]:
train_val_trial_path

In [ ]:
val_data_path

### Make a list of all selected image ids by looping through the label paths: 

In [ ]:
round_counter = 0
selected_image_ids = []
for label_path in label_paths:
    #print(label_path)
    last_part = label_path.split('_')[-1]
    image_id = int(last_part.split('.')[0])
    selected_image_ids.append(image_id)
    round_counter += 1
    #if round_counter > 2:
     #   break

In [ ]:
print(len(selected_image_ids))
print(selected_image_ids[0:7])


### Shuffle selected image ids and subdivide them into training and validation set:

In [ ]:
import random

def split_shuffle(string_list, split_ratio=0.8):
    # Shuffle the list in place
    random.shuffle(string_list)
    
    # Calculate split point
    split_point = int(len(string_list) * split_ratio)
    
    # Split the list
    train_set = string_list[:split_point]
    test_set = string_list[split_point:]
    
    return train_set, test_set

In [ ]:
print(len(selected_image_ids))
train_ids, val_ids = split_shuffle(selected_image_ids)
print(len(train_ids))
print(len(val_ids))

### Get a list of all image files:

In [ ]:
all_file_list = os.listdir(data_path)
image_file_list = []
for filename in all_file_list:
    file_extension = filename.split('.')[-1]
    if file_extension == 'jpg':
        image_file_list.append(filename)
    
print(len(image_file_list))

image_file_list[0:7]

### Loop through image file list and label_paths list and move files to the training or validation folder according to their id:

In [ ]:
train_imgages_path

In [ ]:
train_labels_path

In [ ]:
train_ids[0:7]

In [ ]:
val_ids[0:7]

In [ ]:
image_file_list.sort(key=len, reverse=True)
for file in image_file_list:
    print(len(file))
    print(file)
    break

In [ ]:
label_paths[0:2]

In [ ]:
import shutil

In [ ]:
dist_train_val(image_file_list, train_ids, val_ids, data_path, 
               train_imgages_path, val_imgages_path)
dist_train_val(label_paths, train_ids, val_ids, yolo_path, 
                           train_labels_path, val_labels_path, full_path=True)


### Remove content from meta data files referring to badly labelled images (the word referring to object class written on signs or inside of rooms)

In [ ]:
from source.visual_genome_data import get_file_by_id

def clear_yolo_metadata_by_id(data_path, identifier, id_end=True):
   """
   Clear YOLO metadata file by identifier - makes it empty (removes all bounding boxes)
   
   Args:
       data_path: Path to directory containing .txt files
       identifier: Integer identifier to search for
       id_end: If True (default), select file with ID at end only.
               If False, select file with ID surrounded by underscores.
   """

   
   # Get all matching files
   txt_files = get_file_by_id(data_path, identifier, '.txt')
   
   if not txt_files:
       print(f"No .txt file found with identifier {identifier}")
       return False
   
   # Filter based on id_end parameter
   if id_end:
       # Select only files where ID is at the end
       filtered_files = [f for f in txt_files if f.endswith(f'_{identifier}.txt')]
   else:
       # Select only files where ID is surrounded by underscores
       filtered_files = [f for f in txt_files if f'_{identifier}_' in f]
   
   if not filtered_files:
       pattern_type = "at end" if id_end else "with underscores"
       print(f"No .txt file found with identifier {identifier} {pattern_type}")
       return False
   
   # Overwrite with empty content
   file_path = os.path.join(data_path, filtered_files[0])
   with open(file_path, 'w') as f:
       pass
   
   print(f"Cleared metadata file: {filtered_files[0]}")
   return True

### Convert images to grey scale images

In [ ]:
import cv2
import os
from pathlib import Path

def convert_dataset_to_grayscale(input_dir, output_dir):
    """Convert all images in dataset to grayscale"""
    os.makedirs(output_dir, exist_ok=True)
    
    for img_path in Path(input_dir).glob('*.jpg'):
        # Read image
        img = cv2.imread(str(img_path))
        
        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Convert back to 3-channel (YOLO expects 3 channels)
        gray_3channel = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
        
        # Save
        output_path = os.path.join(output_dir, img_path.name)
        cv2.imwrite(output_path, gray_3channel)


In [ ]:
train_data_path 

In [ ]:
val_data_path

In [ ]:
train_data_path

In [ ]:
# Convert your training images
convert_dataset_to_grayscale(train_images_path, train_images_grey_path)
convert_dataset_to_grayscale(val_images_path, val_images_grey_path)

In [ ]:
os.getcwd()

In [ ]:
test_files_path = root_path / 'test_files'
test_files_grey_path = root_path / 'test_files_grey'

In [ ]:
convert_dataset_to_grayscale(test_files_path, test_files_grey_path)

### Adapt brightness and contrast of grey scale images to make them look old:

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random
import os

In [ ]:
def simulate_specific_old_effects(image_path, output_path):
    """
    Apply specific effects that match your old photos.
    Adjust these based on what you observe in your test images.
    """
    img = Image.open(image_path).convert('RGB')
    
    # Heavy JPEG compression (very low quality)
    img.save('temp.jpg', 'JPEG', quality=15)
    img = Image.open('temp.jpg')
    os.remove('temp.jpg')
    
    # Significant brightness reduction
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(0.8)
    
    # Low contrast
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(0.9)
    
    # Add significant noise
    img_array = np.array(img)
    noise = np.random.normal(0, 0.1, img_array.shape).astype(np.uint8)
    img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    img = Image.fromarray(img_array)
    
    # Strong blur
    img = img.filter(ImageFilter.GaussianBlur(radius=1.5))
    
    img.save(output_path, 'JPEG', quality=85) 


In [ ]:
def process_training_dataset_spec(input_dir, output_dir, augmentation_ratio=0.5):
    """
    Process a directory of training images to simulate old photo effects.
    
    Args:
        input_dir: Directory with original images
        output_dir: Directory to save processed images
        augmentation_ratio: Fraction of images to augment (0.5 = 50%)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif'))]
    
    for img_file in image_files:
        input_path = os.path.join(input_dir, img_file)
        
        # Always copy original
        original_output = os.path.join(output_dir, img_file)
        #img = Image.open(input_path)
        #img.save(original_output)
        

        # Create filename for augmented version
        name, ext = os.path.splitext(img_file)
        aug_filename = f"{name}_aged{ext}"
        aug_output = os.path.join(output_dir, aug_filename)
        #aug_output = os.path.join(output_dir, img_file)
        
        # Apply aging effects with random intensity
        intensity = random.uniform(0.3, 0.8)
        simulate_specific_old_effects(input_path, aug_output)
    
    print(f"Processed {len(image_files)} images in {input_dir}")

In [ ]:
train_images_aged_path = train_val_trial_path / 'train_grey/images_aged'
val_images_aged_path = train_val_trial_path / 'val_grey/images_aged'

In [ ]:
process_training_dataset_spec(train_images_grey_path, train_images_aged_path)
process_training_dataset_spec(val_images_grey_path, val_images_aged_path)

### Move all grey images back to train and val folders (overwriting the coloured images)

In [ ]:
grey_image_list = os.listdir(train_images_grey_path)
for image in grey_image_list:
    grey_image_path = train_images_grey_path / image
    dest_grey_image_path = train_images_path / image
    shutil.copy(grey_image_path, dest_grey_image_path)

In [ ]:
grey_image_list = os.listdir(val_images_grey_path)
for image in grey_image_list:
    grey_image_path = val_images_grey_path / image
    dest_grey_image_path = val_images_path / image
    shutil.copy(grey_image_path, dest_grey_image_path)

In [ ]:
grey_image_list = os.listdir(test_files_grey_path)
for image in grey_image_list:
    grey_image_path = test_files_grey_path / image
    dest_grey_image_path = test_files_path / image
    shutil.copy(grey_image_path, dest_grey_image_path)

In [ ]:
from source.visual_genome_aged_effect import simulate_specific_old_effects
from source.visual_genome_aged_effect import process_training_dataset_spec
from source.visual_genome_aged_effect import copy_with_new_id
from source.visual_genome_aged_effect import add_new_id_img_meta

In [ ]:
train_images_aged_path

### Get identifiers of training and validation images:

In [ ]:
train_img_ids = get_image_ids(str(train_images_path))
val_img_ids = get_image_ids(str(val_images_path))
train_max_id = max(train_img_ids)
val_max_id = max(val_img_ids)
max_id = max([train_max_id, val_max_id])
max_id

### Create and add new identifiers to aged versions of images; move files to train and val folder:

#### The aged image versions are added to the original grey scale images, so that for every image there is an original and a grey scale version

In [ ]:
file_extensions = ['.jpg', '.txt']
tag = 'aged'
add_new_id_img_meta(train_images_aged_path, train_labels_path, 
                    train_images_path, train_labels_path, 
                    train_img_ids, max_id, tag, file_extensions)

In [ ]:
train_img_ids = get_image_ids(str(train_images_path))
val_img_ids = get_image_ids(str(val_images_path))
train_max_id = max(train_img_ids)
val_max_id = max(val_img_ids)
max_id = max([train_max_id, val_max_id])
max_id

In [ ]:
file_extensions = ['.jpg', '.txt']
tag = 'aged'

add_new_id_img_meta(val_images_aged_path, val_labels_path, 
                    val_images_path, val_labels_path, 
                    val_img_ids, max_id, tag, file_extensions)

### Copy yaml file to training folder:

In [ ]:
yaml_path

In [ ]:
yolo_yaml_path = train_val_trial_path / yaml_file_name
yolo_yaml_path

In [ ]:
shutil.copy(yaml_path, yolo_yaml_path)

In [ ]:
itercounter = 0
#img_file_test_list = ['visual_genome_1116.jpg']
#for file in image_file_list:
for file in img_file_test_list:
    print(file)
    img_id = int(file.split('_')[-1].split('.')[0])
    print(img_id)
    print(type(img_id))
    if img_id in train_ids:
        print('train')
        source_path_img = data_path / file
        destination_path_img = train_imgs / file
        shutil.copy2(source_path_img, destination_path_img)
        
    elif img_id in val_ids:
        print('val')
        source_path = data_path / file
        destination_path = val_imgs / file
        shutil.copy2(source_path, destination_path)
    else:
        print('no')
    itercounter += 1
    if itercounter > 70:
        break

In [ ]:
round_counter = 0
for img_id in image_id_list:
    print(img_id)
    round_counter += 1
    if round_counter > 2:
        break

In [ ]:
desired_objects = ['tree', 'trees', 'forest', 'woods', 'mountain', 'mountains', 'building', 'house', 
                   'church', 'city', 'village', 'lake', 'river', 'stream', 'glacier', 'water body', 
                   'watercourse', 'water', 'waters']
class_map = create_class_mapping_from_list(desired_objects)


In [ ]:
class_map

In [ ]:

yaml_path = yolo_path/'class_map.yaml'

In [ ]:

save_class_map_to_yaml(class_map, yaml_path)

In [ ]:
class_map = read_yaml_to_class_map(yaml_path)

In [ ]:
class_map

In [ ]:
#image_dir = '/Users/stephanehess/Documents/CAS_AML/dias_digit_project/visual_genome_data/'
#output_dir = root_path/'visual_genome_yolo'

label_path = convert_single_image_to_yolo(objects[0], class_map, data_path, yolo_path)

In [ ]:
all_files = os.listdir(data_path)
image_file_list = [f for f in all_files if f.lower().endswith('.jpg')]
image_file_list.sort()
image_file_list[0:5]

In [ ]:
round_counter = 0
for filename in image_file_list:
    file_path = data_path/filename
    print(file_path)
    round_counter += 1
    if round_counter > 5: 
        break

In [ ]:
len(image_file_list)

In [ ]:
data_path

In [ ]:
test_path = root_path/'test_visual_genome'

In [ ]:
image_id_list = get_image_ids(data_path)
len(image_id_list)

In [ ]:
image_id_list.sort()

In [ ]:
print(image_id_list[0:3])
print(image_id_list[0])

In [ ]:
iterations = 0
for idx in list(range(0, len(objects))):
    print('\n')
    print(idx)

    object_idx = objects[idx]

    print('image_id')
    img_id = object_idx['image_id']
    print(img_id)
    
    if img_id in image_id_list:

        label_path = convert_single_image_to_yolo(object_idx, class_map, data_path, yolo_path)
        print(label_path)
    iterations += 1
    if iterations > 2:
        break

In [ ]:
import cv2
import numpy as np
import yaml
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os

def visualize_yolo_annotations(image_path, label_path, yaml_path):
    """
    Visualize an image with its YOLO format bounding boxes.
    
    Args:
        image_path: Path to the image file
        label_path: Path to the YOLO format text file with bounding boxes
        yaml_path: Path to the YAML file with class mappings
    """
    # Load the class mappings from YAML file
    with open(yaml_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    
    class_names = yaml_data['names']
    print(label_path)
    # Load the image
    print(image_path)
    image = Image.open(image_path)
    img_width, img_height = image.size
    
    # Convert PIL Image to numpy array for matplotlib
    image_np = np.array(image)
    
    # Create a figure and axis
    fig, ax = plt.subplots(1, figsize=(12, 9))
    
    # Display the image
    ax.imshow(image_np)
    
    # Define colors for bounding boxes (one for each class)
    # Using a colormap to generate distinct colors
    cmap = plt.cm.get_cmap('hsv', len(class_names))
    colors = [cmap(i) for i in range(len(class_names))]
    
    # Load and draw bounding boxes from the label file
    with open(label_path, 'r') as f:
        for line in f:
            data = line.strip().split()
            print(data)
            if len(data) != 5:  # YOLO format has 5 values per line
                continue
                
            class_id = int(data[0])
            x_center = float(data[1]) * img_width
            y_center = float(data[2]) * img_height
            width = float(data[3]) * img_width
            height = float(data[4]) * img_height
            
            # Calculate top-left corner for rectangle
            x_min = x_center - (width / 2)
            y_min = y_center - (height / 2)
            
            # Create a rectangle patch
            rect = patches.Rectangle(
                (x_min, y_min), width, height, 
                linewidth=2, 
                edgecolor=colors[class_id], 
                facecolor='none'
            )
            
            # Add the rectangle to the plot
            ax.add_patch(rect)
            
            # Add class label text above the bounding box
            class_name = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
            plt.text(
                x_min, y_min - 5, 
                class_name,
                color=colors[class_id], 
                fontsize=12, 
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=0)
            )
    
    plt.axis('off')  # Hide axes
    plt.tight_layout()
    
    return fig, ax

In [ ]:

round_counter = 0
for filename in image_file_list:
    image_path = image_dir + filename
    print(image_path)
    last_part = image_path.split('_')[-1]
    img_id_str = last_part.split('.')[0]
    img_id = int(img_id_str)
    img_id_minus_1 = img_id-1
    #print(img_id)
    #print(objects[img_id_minus_1]['image_id'])
    print(image_path.split('.')[0])
    file_path_no_ext = image_path.split('.')[0]
    print(file_path_no_ext)
    label_path_no_ext = file_path_no_ext.replace('visual_genome_data', 'visual_genome_yolo')
    print(label_path_no_ext)
    label_path = label_path_no_ext + '.txt'
    
    visualize_yolo_annotations(image_path, label_path, yaml_path)
    
    
    round_counter += 1
    if round_counter > 5: 
        break